In [2]:
import pandas as pd

In [233]:
data = pd.read_csv('./data_files/clean_data_with_orgs.csv', lineterminator='\n',index_col=0)

In [267]:
#  data
data = data[data['Organizations'].apply(lambda x: 'GME' in x)]


385     False
630     False
915     False
5       False
35      False
        ...  
9660    False
9832    False
9835    False
9862    False
9879    False
Name: subreddit, Length: 17350, dtype: bool

In [261]:
reddit_groups = data.groupby(['subreddit'])

In [276]:
investing_gme = reddit_groups['created'].value_counts().loc['stocks']
mentions['investing'] = investing_gme
mentions.sort_index(axis=0)

,investing
created,
2021-05-17,3
2021-05-18,12
2021-05-19,60
2021-05-20,55
2021-05-21,14
2021-05-22,16
2021-05-23,27
2021-05-24,45
2021-05-25,110


In [274]:
subreddits = ['wallstreetbets', 'stocks', 'investing', 'StockMarket','pennystocks','algotrading', 'RobinHood']
def df_mention_counts(ticker):
    mentions = pd.DataFrame()
    for subreddit in subreddits:
        tickers = data[data['Organizations'].apply(lambda x: ticker in x)]
        count = reddit_groups['created'].value_counts().loc[subreddit]
        mentions[''+subreddit+''] = count
    return mentions

In [279]:
#TICKERS = ['AMC','GME','PLTR','AAPL','SPCE','AMZN','FORD','AMD','TSLA','SEC','ATH','NOK','OTM','ITM','AI','COIN','VIX','BB','RBLX','HODL']
#for ticker in TICKERS:
#    return df_mention_counts(TICKER)
    



In [284]:
a = df_mention_counts('GME')
a.sort_index(axis=0)


,wallstreetbets,stocks,investing,StockMarket,pennystocks,algotrading,RobinHood
created,,,,,,,
2021-05-15,2,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-16,73,NaN,NaN,2.0,NaN,NaN,NaN
2021-05-17,355,3.0,11.0,2.0,NaN,NaN,NaN
2021-05-18,1700,12.0,19.0,24.0,NaN,NaN,2.0
2021-05-19,820,60.0,16.0,31.0,NaN,NaN,4.0
2021-05-20,451,55.0,3.0,12.0,NaN,NaN,NaN
2021-05-21,616,14.0,7.0,15.0,NaN,NaN,NaN
2021-05-22,427,16.0,1.0,4.0,NaN,2.0,NaN
2021-05-23,510,27.0,7.0,42.0,NaN,1.0,NaN
